In [7]:
from pathlib import Path
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from data_processing.data_processing import *

from sklearn import preprocessing
from sklearn import linear_model
from sklearn import metrics
from sklearn.pipeline import Pipeline

In [2]:
MAIN_PATH = Path(os.getcwd()).parent

# Load data

In [3]:
org_df = pd.read_csv(os.path.join(MAIN_PATH, "data", "data.csv"))
org_df = load(org_df)
org_df

,001fcx00211.pv,001fcx00221.pv,001fcx00231.pv,001fcx00241.pv,001fir01307.daca.pv,001fir01308.daca.pv,001fir01309.daca.pv,001fir01310.daca.pv,001fir01311.daca.pv,001fir01312.daca.pv,...,001uxm0rf02.daca.pv,001uxm0rf03.daca.pv,037tix00254.daca.pv,037tix00264.daca.pv,prazonka_fe,prazonka_s,prob_corg,prob_fe,prob_s,temp_zuz
2020-09-30 22:00:00+00:00,56.729077,54.724422,11.966905,22.493207,108.702362,112.280388,110.013796,108.761583,110.604901,104.500427,...,92.024078,92.405281,24.665309,24.526161,4.48,8.98,8.6,4.55,9.87,1297.0
2020-09-30 22:01:00+00:00,54.771942,54.734675,12.026410,22.593412,108.846612,112.334600,109.906645,108.879083,110.644046,104.371977,...,92.024206,92.405239,24.663982,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:02:00+00:00,54.695816,54.774163,11.953671,22.428933,108.827640,112.362508,109.970932,108.844765,110.621665,104.387912,...,92.024334,92.405196,24.662656,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:03:00+00:00,54.154394,54.985713,12.052065,22.335388,108.623473,112.386061,109.836948,108.717994,110.464245,104.418573,...,92.024462,92.405153,24.661329,24.526161,4.48,8.98,8.6,4.55,9.87,NaN
2020-09-30 22:04:00+00:00,54.693184,54.490742,12.027310,22.412620,108.746784,112.396792,109.966910,108.978605,110.603660,104.464328,...,92.024590,92.405110,24.660003,24.525475,4.48,8.98,8.6,4.55,9.87,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-31 22:55:00+00:00,57.882366,58.080133,10.056829,30.486496,101.111263,98.338021,102.332506,101.967828,100.651891,97.071359,...,98.121880,98.225985,24.460338,24.320295,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:56:00+00:00,57.953225,57.931008,9.935828,29.830075,101.034501,98.362465,102.332450,101.981035,100.575884,97.066000,...,98.121319,98.226027,24.464324,24.322539,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:57:00+00:00,58.186232,57.291012,10.014864,29.350514,101.251020,98.353221,102.312023,102.017297,100.621084,97.223686,...,98.120757,98.226070,24.468310,24.324782,6.18,8.89,7.8,6.65,12.47,NaN
2022-01-31 22:58:00+00:00,59.016838,58.014325,10.048125,29.496899,101.073264,98.354741,102.336123,101.967655,100.596174,97.050953,...,98.120196,98.226113,24.472296,24.327025,6.18,8.89,7.8,6.65,12.47,NaN


In [4]:
df = pd.DataFrame(index=org_df.index)

In [5]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = org_df.columns[org_df.columns.str.contains('001tix')].values
df['TIX1'] = org_df[TIX1].mean(axis='columns')
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = org_df.columns[org_df.columns.str.contains('tir')].values
df['TIR'] = org_df[TIR].mean(axis='columns')
# dodanie reg nadawy koncentratu liw i uśrednienie
LIW = org_df.get(['001fcx00211.pv', '001fcx00221.pv'])
df['LIW'] = LIW.mean(axis='columns')
# dodanie pozostałych reg nadawy
df['001fcx00231.pv'] = org_df['001fcx00231.pv']
df['001fcx00241.pv'] = org_df['001fcx00241.pv']
# sumaryczna moc cieplna
NIR = org_df.columns[org_df.columns.str.contains('nir')].values
df[NIR] = org_df[NIR]
# dodanie temp żużla
df["TEMP_ZUZ"] = org_df["temp_zuz"]
df = df.dropna(axis=0)
print(df.shape)
df

(11377, 7)


,TIX1,TIR,LIW,001fcx00231.pv,001fcx00241.pv,001nir0szr0.daca.pv,TEMP_ZUZ
2020-09-30 22:00:00+00:00,418.990965,29.086390,55.726750,11.966905,22.493207,14.362428,1297.0
2020-09-30 23:00:00+00:00,418.911544,29.000256,55.318554,11.544775,22.702118,14.978720,1295.0
2020-10-01 00:00:00+00:00,418.820059,29.021386,54.830731,11.053689,20.013150,15.199544,1303.0
2020-10-01 00:30:00+00:00,418.777284,29.064571,55.112358,10.980221,19.902115,14.815123,1302.0
2020-10-01 01:00:00+00:00,418.735368,29.092727,55.378319,11.057164,21.069357,14.617263,1303.0
...,...,...,...,...,...,...,...
2022-01-31 18:00:00+00:00,418.284737,27.694067,59.040197,10.019018,32.147630,15.607105,1300.0
2022-01-31 19:00:00+00:00,418.138199,27.760225,58.903828,10.013093,31.719701,14.846545,1305.0
2022-01-31 20:00:00+00:00,418.012402,27.761558,59.333633,10.064700,30.158128,15.967954,1304.0
2022-01-31 21:00:00+00:00,417.939267,27.704117,58.256581,9.989189,28.000376,16.147269,1308.0


# Split dataset

In [6]:
train, val, test = split(df)
X_train, X_val, X_test = train.drop(["TEMP_ZUZ"], axis=1), val.drop(["TEMP_ZUZ"], axis=1), test.drop(["TEMP_ZUZ"],
                                                                                                     axis=1)
y_train, y_val, y_test = train["TEMP_ZUZ"], val["TEMP_ZUZ"], test["TEMP_ZUZ"]

# Ridge regression

In [14]:
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
alphas = [.1, .5, 1]

for solver in solvers:
    for alpha in alphas:
        ridge_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                                ('ridge', linear_model.Ridge(alpha=alpha, solver=solver))])
        ridge_model.fit(X_train, y_train)
        mse = metrics.mean_squared_error(y_val, ridge_model.predict(X_val))
        print(solver, alpha, mse, ridge_model.named_steps['ridge'].coef_)

svd 0.1 74.99438774660148 [1.04762165 0.30772092 0.57339082 0.42184525 0.72730472 0.96905781]
svd 0.5 74.99446192014945 [1.04753609 0.3077011  0.57341069 0.42174708 0.72727997 0.96900992]
svd 1 74.99455470419592 [1.04742917 0.30767634 0.5734355  0.42162442 0.72724903 0.96895008]
cholesky 0.1 74.99438774660148 [1.04762165 0.30772092 0.57339082 0.42184525 0.72730472 0.96905781]
cholesky 0.5 74.99446192014945 [1.04753609 0.3077011  0.57341069 0.42174708 0.72727997 0.96900992]
cholesky 1 74.99455470419595 [1.04742917 0.30767634 0.5734355  0.42162442 0.72724903 0.96895008]
lsqr 0.1 74.98217603875321 [1.04430362 0.30682037 0.57741248 0.42256738 0.72013707 0.97505024]
lsqr 0.5 74.98225787437232 [1.04421995 0.30680109 0.57742973 0.42246901 0.72011657 0.97499904]
lsqr 1 74.98236022742431 [1.04411539 0.30677699 0.57745127 0.42234612 0.72009095 0.97493506]
sparse_cg 0.1 74.99438774660148 [1.04762165 0.30772092 0.57339082 0.42184525 0.72730472 0.96905781]
sparse_cg 0.5 74.99446192014945 [1.0475360

# Ridge Regression + Polynomial Features

In [15]:
solvers = ['svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']
alphas = [.1, .5, 1]
degrees = [2, 3, 4]

print("solver, alpha, degree, mse")
for solver in solvers:
    for alpha in alphas:
        for degree in degrees:
            ridge_model = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=degree)),
                                    ('normalization', preprocessing.StandardScaler()),
                                    ('ridge', linear_model.Ridge(alpha=alpha, solver=solver))])
            ridge_model.fit(X_train, y_train)
            mse = metrics.mean_squared_error(y_val, ridge_model.predict(X_val))
            print(solver, alpha, degree, mse)

solver, alpha, degree, mse
svd 0.1 2 68.40305457254732
svd 0.1 3 64.70314672421581
svd 0.1 4 66.40785343203846
svd 0.5 2 68.45935609599246
svd 0.5 3 65.14348801192071
svd 0.5 4 64.92807359911265
svd 1 2 68.40122985712314
svd 1 3 65.75674701410641
svd 1 4 64.7301667617553
cholesky 0.1 2 68.40305457251513
cholesky 0.1 3 64.70314672400585
cholesky 0.1 4 66.40785343217476
cholesky 0.5 2 68.4593560959898
cholesky 0.5 3 65.14348801187221
cholesky 0.5 4 64.92807359911653
cholesky 1 2 68.4012298571233
cholesky 1 3 65.75674701407938
cholesky 1 4 64.73016676175963
lsqr 0.1 2 69.15252091750936
lsqr 0.1 3 70.58312292246853
lsqr 0.1 4 70.55841548226576
lsqr 0.5 2 68.91371426289231
lsqr 0.5 3 70.53640775546668
lsqr 0.5 4 70.54878465440278
lsqr 1 2 68.66669508652821
lsqr 1 3 70.4800810232168
lsqr 1 4 70.53690786643219
sparse_cg 0.1 2 69.25780058230411
sparse_cg 0.1 3 64.81195902632635
sparse_cg 0.1 4 64.07335495602491
sparse_cg 0.5 2 69.3211490609048
sparse_cg 0.5 3 66.41472399779049
sparse_cg 0.5 4 

# SGD Regression

In [16]:
penalties = ['l1', 'l2', 'elasticnet']

print("penalty, mse")
for penalty in penalties:
    sgd_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                      ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty=penalty, learning_rate='adaptive'))])
    sgd_model.fit(X_train, y_train)
    mse = metrics.mean_squared_error(y_val, sgd_model.predict(X_val))
    print(penalty, mse)

penalty, mse
l1 74.98970510241956
l2 74.97676232990167
elasticnet 74.97703343689182


# SGD Regression + Polynomial

In [18]:
penalties = ['l1', 'l2', 'elasticnet']
degrees = [2, 3, 4]

print("penalty, degree, mse")
for penalty in penalties:
    for degree in degrees:
        sgd_model = Pipeline([('poly', preprocessing.PolynomialFeatures(degree=degree)),
                            ('normalization', preprocessing.StandardScaler()),
                          ('sgd', linear_model.SGDRegressor(max_iter=100000, tol=1e-3, penalty=penalty, learning_rate='adaptive'))])
        sgd_model.fit(X_train, y_train)
        mse = metrics.mean_squared_error(y_val, sgd_model.predict(X_val))
        print(penalty, degree, mse)

penalty, degree, mse
l1 2 69.44432039551894
l1 3 19439.303013506284
l1 4 6.452432413289027e+18
l2 2 67.98633213844053
l2 3 87.37746501525875
l2 4 7.743630883527502e+18
elasticnet 2 67.76832106532045
elasticnet 3 10436253.497692186
elasticnet 4 5.082672738781665e+18


# ElasticNet Regression

In [17]:
elastic_model1 = Pipeline([('normalization', preprocessing.StandardScaler()),
                           ('elasitc', linear_model.ElasticNet(max_iter=1000))])
elastic_model1 = elastic_model1.fit(X_train, y_train)
metrics.mean_squared_error(y_val, elastic_model1.predict(X_val))

79.39085001063494

# BayesianRidge

In [19]:
bay_model = Pipeline([('normalization', preprocessing.StandardScaler()),
                        ('bayessian', linear_model.BayesianRidge(n_iter=1000))])
bay_model = bay_model.fit(X_train, y_train)
metrics.mean_squared_error(y_val, bay_model.predict(X_val))

75.01047167885115